# DP Aggregate Seeded Synthesizer
DP Aggregate Seeded synthesizer is a differentially private synthesizer that relies on DP Marginals to build synthetic data. It will compute DP Marginals (called aggregates) for your dataset using an specified `reporting length`, and synthesize data based on the computed aggregated counts.

> Aggregates will be computed for all lengths of attribute combination up to and including the `reporting length`.

## 1. Overview

### 1.1. Aggregate data generation with DP
Let's consider the following input as example:

| A | B | C |
| -- | -- | -- |
| a1 | b1 | c1 |
| a1 | b2 | c1 |
| a2 |    | c2 |
| a2 | b2 | c1 |
| a1 | b2 |    |

The input data is assumed to be categorical and the domain will be inferred from the input dataset:

- `A` possible values are `a1,a2`
- `B` possible values are `b1,b2`
- `C` possible values are `c1,c2`

For a `reporting length=2`, the aggregates in the dataset above could be:

- 1-counts
    - `A:a1`: 3 + NOISE
    - `A:a2`: 2 + NOISE
    - `B:b1`: 1 + NOISE
    - `B:b2`: 3 + NOISE
    - `C:c1`: 3 + NOISE
    - `C:c2`: 1 + NOISE

- 2-counts:
    - `A:a1;C:c1`: 2 + NOISE
    - `A:a2;B:b2`: 1 + NOISE
    - `B:b1;C:c1`: 1 + NOISE
    - `A:a1;B:b1`: 1 + NOISE
    - `A:a1;B:b2`: 2 + NOISE
    - `B:b2;C:c1`: 2 + NOISE
    - `A:a2;C:c2`: 1 + NOISE
    - `B:b2;C:c2`: 0 + NOISE

Also, some spurious combinations might be created and reported to ensure differentially private guarantees - notice that `B:b2;C:c2` does not exist in the sensitive dataset, but it has been _fabricated_ and added to the output.

Similarly, some attribute combinations might be suppressed. For example, even though `A:a2;C:c1` exists in the sensitive dataset, it has not been reported as an aggregate.

### 1.2. Synthesis
Data will be then synthesized directly from the aggregates computed with differential privacy to produce synthetic data. Which will ensure the same DP guarantees to the synthetic data.

## 2. Imports and global config

In [49]:
import pandas as pd

from sdssynth import init_logger, set_number_of_threads
from sdssynth import Dataset
from sdssynth import DpAggregateSeededParametersBuilder, AccuracyMode, FabricationMode
from sdssynth import DpAggregatedSeededSynthesizer, Dataset

from utils import gen_dataset, ErrorReport

# The library allows the desired log level to be set if wanted
# ('off' || 'error' || 'warn' || 'info' || 'debug' || 'trace')
# init_logger('trace')

# some algorithms have parallel implementations, so the desired number of threads can be set
# (the default is one thread per CPU core)
# set_number_of_threads(2)

## 3. Generating an example data frame with random data

> `gen_dataset` is just an utility to generate some example data (the code for it is in [`utils.py`](./utils.py))

To illustrate the library, let's start by creating an example data frame:

In [50]:
number_of_records_to_generate = 6000

sensitive_df = gen_dataset(number_of_records_to_generate)

sensitive_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,,,3,1,1,1,0,0,0,1
1,1,,2,1,0,0,0,1,1,0
2,1,3,5,0,0,0,0,1,0,0
3,1,1,1,0,1,1,1,1,0,1
4,,1,1,1,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...
5995,1,6,6,0,0,1,1,1,0,0
5996,,5,10,1,1,0,0,1,0,0
5997,,6,8,1,0,1,1,1,1,1
5998,,5,,0,1,1,1,0,0,1


## 4. Creating the sensitive dataset

### 4.1. Creation from constructor

The library uses an internal representation of the data to optimize execution time complexity.

If the data is already in the required raw format, you can call the constructor directly:

In [51]:
sensitive_raw_data = [
    # headers
    ['A', 'B', 'C', 'D'],
    # records
    ['a1', 'b1', 'c1', '0'],
    ['a1', 'b1', 'c2', '0'],
    ['a2', '', 'c2', '1'],
]
sensitive_dataset = Dataset(sensitive_raw_data)

### 4.2. Negative value interpretation

The library distinguishes 'positive' attribute values that indicate the presence of specific sensitive data from 'negative' attribute values that indicate the absence of such data. By default, the integer zero (`0`) and the empty string (`""`) are not taken into account when creating and counting attribute combinations. Any columns where zero values are of interest (and thus sensitive) should be listed as `sensitive_zeros`, so they will be treated the same way as positive values.

> For more parameters see the library documentation.

In [52]:
sensitive_raw_data = [
    # headers
    ['A', 'B', 'C', 'D'],
    # records
    ['a1', 'b1', 'c1', '0'],
    ['a1', 'b1', 'c2', '0'],
    ['a2', '', 'c2', '1'],
]
sensitive_dataset = Dataset(sensitive_raw_data, sensitive_zeros=['D'])

### 4.3. Creating sensitive dataset from a pandas data frame
For convenience, a method is provided to build a dataset from a pandas data frame:

In [53]:
sensitive_dataset = Dataset.from_data_frame(sensitive_df)

## 5. Generating the synthetic data

### 5.1. Defining synthesizer parameters

If you just want to create the synthesizer with default parameters:

In [54]:
synth = DpAggregatedSeededSynthesizer()
print (synth.parameters)

{
  "reporting_length": 3,
  "epsilon": 0.1,
  "delta": null,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon": 0.1,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": null
}


However, this might not produce the optimal output for your dataset and downstream analysis tasks.

To tune the synthesizer parameters, we provide a builder (`DpAggregateSeededParametersBuilder`). Any parameters passed to the builder will override the default values accordingly:

In [55]:
# this explicitly outlines the default parameters
synth = DpAggregateSeededParametersBuilder() \
        .reporting_length(3) \
        .epsilon(6.0) \
        .delta(1 / (2.0 * len(sensitive_df))) \
        .percentile_percentage(99) \
        .percentile_epsilon_proportion(0.01) \
        .accuracy_mode(AccuracyMode.prioritize_large_counts()) \
        .number_of_records_epsilon(0.1) \
        .fabrication_mode(FabricationMode.uncontrolled()) \
        .empty_value("") \
        .weight_selection_percentile(95) \
        .use_synthetic_counts(False) \
        .aggregate_counts_scale_factor(1.0) \
        .build()
print (synth)


{
  "reporting_length": 3,
  "epsilon": 6.0,
  "delta": 0.00008333333333333333,
  "percentile_percentage": 99,
  "percentile_epsilon_proportion": 0.01,
  "sigma_proportions": [
    1.0,
    0.5,
    0.3333333333333333
  ],
  "number_of_records_epsilon": 0.1,
  "threshold": {
    "type": "Adaptive",
    "valuesByLen": {
      "3": 1.0,
      "2": 1.0
    }
  },
  "empty_value": "",
  "use_synthetic_counts": false,
  "weight_selection_percentile": 95,
  "aggregate_counts_scale_factor": 1.0
}


To continue with this example, let's set the parameters we care about for now:

In [56]:
reporting_length = 4

builder = DpAggregateSeededParametersBuilder() \
    .reporting_length(reporting_length) \
    .epsilon(0.2) \
    .accuracy_mode(AccuracyMode.prioritize_large_counts()) \
    .fabrication_mode(FabricationMode.uncontrolled()) \
    .use_synthetic_counts(True)

synth = DpAggregatedSeededSynthesizer(builder.build())

### 5.2 Building the model and synthesizing data

In [57]:
synth.fit(sensitive_dataset)

# here if we do not specify the desired number of samples, the synthesizer will
# use all the available attributes based on the 1-counts to synthesize records
synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

# as an example, let's create a pandas data frame from the raw synthetic data
synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,2,4,2,,,,,,,
5996,2,4,2,,,,,,,
5997,2,4,2,,,,,,,
5998,2,4,2,,,,,,,


## 6. Generating/exporting aggregate data

In [58]:
# generate sensitive aggregates
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')

# export the differentially private aggregates (internal to the synthesizer)
dp_aggregates = synth.get_dp_aggregates(';')

# generate aggregates from the synthetic data
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

# let's take a look at the DP aggregates
list(dp_aggregates.items())[:20]

[('H1:1;H3:6;H4:1;H8:1', 15),
 ('H2:5;H3:9;H6:1;H8:1', 2),
 ('H1:1;H3:7;H6:1', 129),
 ('H2:2;H3:9', 64),
 ('H2:6;H3:9;H8:1', 32),
 ('H10:1;H3:6;H8:1', 97),
 ('H1:2;H2:5;H4:1', 179),
 ('H10:1;H3:5;H4:1;H6:1', 27),
 ('H10:1;H1:1;H3:7;H8:1', 29),
 ('H2:1;H3:2;H6:1;H8:1', 40),
 ('H1:2;H2:6;H3:7;H4:1', 2),
 ('H10:1;H1:2;H5:1;H6:1', 111),
 ('H10:1;H3:7;H5:1', 121),
 ('H3:1;H4:1;H7:1;H8:1', 9),
 ('H1:2;H2:2;H3:5;H9:1', 5),
 ('H3:10;H4:1;H6:1;H7:1', 85),
 ('H10:1;H1:1;H6:1;H8:1', 236),
 ('H1:1;H3:8;H4:1', 44),
 ('H1:1;H7:1;H8:1', 588),
 ('H1:2;H2:2;H3:7;H4:1', 7)]

## 7. Evaluate DP aggregates and DP synthetic data

This section is an example evaluation of both the DP aggregates and the DP synthetic data, as well as the influence of some synthesizer parameters in the DP aggregates and synthetic data.

### 7.1. Evaluating current results

> `ErrorReport` is just an example way to evaluate results (the code for it is [`utils.py`](./utils.py))

- **Count**: mean of the aggregate counts for the given length
- **Error**: mean of the `abs(sensitive_count - dp_aggregated_count)` or `abs(sensitive_count - synthetic_count)`
- **Suppressed %**: percentage of combinations present in the sensitive dataset, but not present in the aggregated/synthetic data
- **Fabricated %**: percentage of combinations that were reported in the aggregated/synthetic data, but do not exist in the sensitive dataset

In [59]:
sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive x DP Aggregates**

In [60]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1377.20 +/- 53.88,0.00 %,0.00 %
1,2,422.89 +/- 49.90,0.96 %,7.59 %
2,3,147.57 +/- 40.73,8.49 %,8.59 %
3,4,55.56 +/- 25.77,37.83 %,4.63 %
4,All,110.34 +/- 32.88,27.83 %,6.16 %


**Sensitive Data vs. DP Synthetic Data**

In [61]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1377.20 +/- 176.76,0.00 %,0.00 %
1,2,422.89 +/- 125.21,0.96 %,7.59 %
2,3,147.57 +/- 71.11,8.60 %,8.09 %
3,4,55.56 +/- 36.70,38.84 %,4.26 %
4,All,110.34 +/- 56.67,28.54 %,5.79 %


### 7.2. Targeting less fabrication

Let's update the current synthesizer parameters to `minimize` fabrication:

In [62]:
synth = DpAggregatedSeededSynthesizer(
    builder \
        .fabrication_mode(FabricationMode.minimize()) \
        .build()
)

synth.fit(sensitive_dataset)

synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,1,,,1,1,1,1,1,1,1
1,1,,,1,1,1,1,1,1,1
2,1,,,1,1,1,1,1,1,1
3,1,,,1,1,1,1,1,1,1
4,1,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,2,3,,,,,,,,
5996,2,3,,,,,,,,
5997,2,3,,,,,,,,
5998,2,3,,,,,,,,


Evaluating again:

In [63]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')
dp_aggregates = synth.get_dp_aggregates(';')
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive Data vs. DP Aggregates**

In [64]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1377.20 +/- 60.68,0.00 %,0.00 %
1,2,422.89 +/- 50.28,10.53 %,0.00 %
2,3,147.57 +/- 61.97,76.96 %,0.00 %
3,4,55.56 +/- 34.83,94.72 %,0.00 %
4,All,110.34 +/- 51.66,84.50 %,0.00 %


**Sensitive Data vs. DP Synthetic Data**

In [65]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1377.20 +/- 501.80,0.00 %,0.00 %
1,2,422.89 +/- 271.59,10.53 %,0.00 %
2,3,147.57 +/- 160.79,76.96 %,0.00 %
3,4,55.56 +/- 51.38,94.72 %,0.00 %
4,All,110.34 +/- 189.05,84.50 %,0.00 %


### 7.3. Prioritize small counts

Let's update the current synthesizer parameters to `prioritize_small_counts`:

In [66]:
synth = DpAggregatedSeededSynthesizer(
    builder \
        .fabrication_mode(FabricationMode.uncontrolled()) \
        .accuracy_mode(AccuracyMode.prioritize_small_counts()) \
        .build()
)

synth.fit(sensitive_dataset)

synthetic_raw_data = synth.sample(len(sensitive_df))
synthetic_dataset = Dataset(synthetic_raw_data)

synthetic_df = Dataset.raw_data_to_data_frame(synthetic_raw_data)
synthetic_df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
0,,,,1,1,1,1,1,1,1
1,,,,1,1,1,1,1,1,1
2,,,,1,1,1,1,1,1,1
3,,,,1,1,1,1,1,1,1
4,,,,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...
5995,2,4,8,,,,,,,
5996,2,4,8,,,,,,,
5997,2,5,9,,,,,,,
5998,2,5,9,,,,,,,


Evaluating again:

In [67]:
sensitive_aggregates = sensitive_dataset.get_aggregates(reporting_length, ';')
dp_aggregates = synth.get_dp_aggregates(';')
synthetic_aggregates = synthetic_dataset.get_aggregates(reporting_length, ';')

sensitive_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in sensitive_aggregates.items()
}
dp_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in dp_aggregates.items()
}
synthetic_aggregates_parsed = {
    tuple(agg.split(';')): count for (agg, count) in synthetic_aggregates.items()
}

**Sensitive Data vs. DP Aggregates**

In [68]:
ErrorReport(sensitive_aggregates_parsed, dp_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1377.20 +/- 13.04,0.00 %,0.00 %
1,2,422.89 +/- 37.17,0.00 %,4.57 %
2,3,147.57 +/- 59.54,15.55 %,6.01 %
3,4,55.56 +/- 42.59,64.56 %,4.62 %
4,All,110.34 +/- 48.61,47.68 %,5.14 %


**Sensitive Data vs. DP Synthetic Data**

In [69]:
ErrorReport(sensitive_aggregates_parsed, synthetic_aggregates_parsed).gen()

,Length,Count +/- Error,Suppressed %,Fabricated %
0,1,1377.20 +/- 306.88,0.00 %,0.00 %
1,2,422.89 +/- 195.43,0.00 %,4.13 %
2,3,147.57 +/- 104.06,15.88 %,5.45 %
3,4,55.56 +/- 56.55,65.77 %,4.33 %
4,All,110.34 +/- 95.98,48.58 %,4.73 %
